In [37]:
#Imports
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd


In [38]:
# Load environment variables from .env file
load_dotenv()

True

In [39]:
#Adjustable weights for aroma
aroma_weight = 0.5

# Placeholder for future commercial enhancements
# ------------------------------------------------
# In future versions, we may introduce dynamic weighting for aroma influence.
# This could allow:
# - Adjusting recommendations based on user engagement data.
# - Sponsored wines to have higher weight in AI-driven suggestions.
# - Adaptive learning to refine preferences over time.
# To implement, consider replacing `aroma_weight` with a dynamic function.
# ------------------------------------------------


In [40]:
# Retrieve the API key
api_key = os.getenv("Gemini_API_Key")
if not api_key:
    raise ValueError("API key not found. Ensure 'Gemini_API_Key' is correctly set in the .env file.")

genai.configure(api_key=api_key)

In [41]:
# Load the wine dataset
nlp_file_path = "../Resources/nlp_3.1_final.csv"
wine_df = pd.read_csv(nlp_file_path)

In [42]:
# Extract unique attribute options
color_options = wine_df["color"].explode().unique().tolist()
aroma_options = wine_df["aromas"].explode().unique().tolist()
flavor_options = wine_df["flavors"].explode().unique().tolist()
body_options = ["Light-bodied", "Medium-bodied", "Full-bodied"] #added for body options may need to be extracted from the dataset


In [43]:
#Finalized Wine Color Options with Descriptions
wine_colors = {
    "Red": "Ranges from light (garnet) to deep (opaque) reds. Full-bodied reds tend to have higher tannins.",
    "White": "Ranges from clear to golden hues, with fuller-bodied whites having a richer color due to oak aging.",
    "Rosé": "Varies from pale salmon to magenta, depending on grape variety and skin contact time.",
    "Orange": "A white wine made with extended skin contact, giving it a deep amber color.",
    "Sparkling": "Can be white, rosé, or even red with effervescence from natural carbonation."
}

In [ ]:
#Aromor Options
#We can farm this data from the dataset during cleanup if we want to use it directly
aroma_options = [
    "Fruity White Wine - Citrus",
    "Fruity White Wine - White Berries",
    "Fruity White Wine - Pome Fruits",
    "Fruity White Wine - Stone Fruits",
    "Fruity White Wine - Tropical Fruits",
    "Fruity White Wine - Botrytized",
    "Fruity Red Wine - Tropical Fruits",
    "Fruity Red Wine - Red Berries",
    "Fruity Red Wine - Stone Fruits",
    "Fruity Red Wine - Fortified",
    "Floral - White Flowers",
    "Floral - Colored Flowers",
    "Vegetal - Vegetables",
    "Vegetal - Fresh Herbs",
    "Vegetal - Dried Herbs",
    "Vegetal - Leaves",
    "Maturation in Oak Barrel - Woods",
    "Maturation in Oak Barrel - Nuts",
    "Maturation in Oak Barrel - Spices",
    "Maturation in Oak Barrel - Toasted"
]

In [45]:
#Price Selection (Dropdown with explicit ranges)
price_options = {
    "$1 - $9.99": "Budget-friendly wines, great for casual drinking.",
    "$10 - $99.99": "Standard quality wines, suitable for most occasions.",
    "$100 - $999.99": "Premium and high-quality wines for special occasions.",
    "$1000+": "Rare, collectible, and investment-grade wines."
}

In [46]:
# Wine Recommendation Function
def get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range):
    """First searches the dataset for a wine recommendation. If no match, use Gemini AI."""
    
    # Filter dataset based on user inputs
    filtered_wines = wine_df[
        wine_df["color"].apply(lambda x: wine_color in x) &
        wine_df["flavors"].apply(lambda x: flavor_profile in x) &
        wine_df["aromas"].apply(lambda x: aroma in x)
    ]
    
  # If we find a matching wine in the dataset
    if not filtered_wines.empty:
        recommended_wine = filtered_wines.sample(1).iloc[0]
        wine_recommendation = f"Recommended Wine: {recommended_wine['color']} - Aromas: {recommended_wine['aromas']} - Flavors: {', '.join(recommended_wine['flavors'].strip('[]').replace("'", "").split(','))} - Body: {body} - Price Range: {price_range}"
    else:
        # If no exact match, use Gemini AI
        prompt = f"""
        You are a wine expert. Based on the following attributes, recommend a wine:
        - Wine Color: {wine_color}
        - Flavor Profile: {flavor_profile}
        - Aroma: {aroma}
        - Sweetness Level: {sweetness}
        - Body: {body}
        - Price Range: {price_range}
        Provide the best wine recommendation.
        """
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        wine_recommendation = response.text
    
    return wine_recommendation



In [47]:
#UI Function Placeholder(obsolte)
# def recommend_wine(wine_color, aroma, sweetness, price_range):
#     description = f"You selected a {wine_color} wine. {wine_colors[wine_color]}\n"
#     description += f"Aroma Preference: {aroma}\n"
#     description += f"Sweetness Level: {sweetness} (0 = Dry, 10 = Very Sweet)\n"
#     description += f"Price Range: {price_range} - {price_options[price_range]}\n"
#     description += "\nGenerating best wine recommendation based on your preferences..."
#     return description

In [48]:
# Build Gradio UI
demo = gr.Interface(
    fn=get_wine_recommendation,
    inputs=[
        gr.Dropdown(list(wine_colors.keys()), label="Wine Color"),
        gr.Dropdown(aroma_options, label="Aroma Preference"),
        gr.Dropdown(flavor_options, label="Flavor Profile"),
        gr.Slider(0, 10, step=1, label="Sweetness Level (0 = Dry, 10 = Very Sweet)"),
        gr.Dropdown(body_options, label="Wine Body"),
        gr.Dropdown(list(price_options.keys()), label="Price Range")
    ],
    outputs=gr.Textbox(label="Wine Recommendation")
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
